This is a example how i think you can exploit the database set

Giving two symptoms and coming up with a ranking of possible diseases
Can someone improve this ?

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output

print(check_output(["ls", "../input"]).decode("utf8"))
syd=pd.read_csv('../input/sym_dis_matrix.csv')
dia=pd.read_csv('../input/dia_3.csv')
sym=pd.read_csv('../input/sym_3.csv')
#
A=syd
A=A.append(A)
B=A.drop(['eye'],axis=1)
C=B
#You can merge the symptoms and the diagnosis
#A=pd.merge(sym,syd,how='inner',right_on='eye',left_on='chief_complaint_id')
#B=pd.merge(A,dia,how='inner',left_on='_id',right_on='id')
C.index.name='eye'
C_df=pd.DataFrame(C)
dia.reset_index(level=0, inplace=True)
print(len(dia)-1, 'nr of disease')
print(len(sym)-1, 'nr of symptoms')

In [ ]:
#this is the core function, SVD searching for singularity
from scipy.cluster.vq import whiten
C=whiten(C_df)
#the eye column moet gedropt worden
#print(C)
from numpy.linalg import inv
U,s,V=np.linalg.svd(C,full_matrices=False)
# reconstruct
S=np.diag(s)

# recude the columns and see the influence
reductie=30
print(s[:reductie].sum(), 'explained variability of total ',s.sum(),s[:reductie].sum()/s.sum()*100," % variation explained")
#reduce the number of columns and see the influence on the ranking
S=S[0:reductie,0:reductie]
iS=inv(S)
US=np.dot(U[:,0:reductie],iS)

# A fill up with US matrix
US_df=pd.DataFrame(data=US)
#print(A.head())
print(A.shape)
#print( 'UMatrix',U)
print(U.shape)
#print('VMatrix',V)
print(V.shape)
#print('USMatrix',US)
print(US.shape)
# with this simple math i know all the relations between all the symptoms and diseases
#

In [ ]:
#visualizing the data matrixes

import matplotlib.pylab as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(U, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(S, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()


import matplotlib.pylab as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(V, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(US, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()


import matplotlib.pylab as plt
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(iS, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
plt.imshow(US_df, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
plt.show()

Here you can change the symptoms and select the values in the similarity matrix

In [ ]:
#sym.info()
# CHANGE THE SYMPToMS

#looking up the symptoms id's for specific symptom names
#Qsym= ( sym.ix[(sym['symptom']=='Eye redness') | (sym['symptom'] == 'Eye pain (Irritation)')   ] )
Qsym=(sym.loc[(sym['symptom']=='Lower abdominal pain')| (sym['symptom'] == 'Shortness of breath')| (sym['symptom'] == 'Throat pain') ])
#| (sym['symptom'] == 'Alcohol abuse')| (sym['symptom'] == 'Anxiety (Nervousness)') | (sym['symptom'] == 'Skin itching') 
Qsym.index.name='eye'
print(Qsym)
#Qsymi=Qsym.ix[0]
print(Qsym.iloc[0,:].name)
Qsym2=pd.DataFrame({'eye': []})
#print(A) #print(US_df)
#***** select Qsym2 coordinates in reduced vector space
for xx in range (len(Qsym)):
    waa=Qsym.iloc[xx,:].name
    print(float(waa-1))
    # looking up Symptom 24,30 in US_df
    Qsy=pd.DataFrame((pd.DataFrame(US_df.loc[waa-1,:reductie-1]).T))
    Qsym2=Qsym2.append(Qsy)
del Qsym2['eye']

Qtemp=Qsym2.sum()*2


#print('USQuerymatrix',Qsym2) #print(Qtemp.shape)
# ******************************************* PRINT LIST OF DISEASES sorted on similarity descending
#print(dia.query('id==4'))
# print all similar diseases
#list diseases
#print(len(V))

#    simild=np.dot(Qtemp,V[0:reductie,xyz])/np.dot(np.abs(Qtemp),np.abs(V[0:reductie,xyz]))*100

dise=(dia[dia['_id']==0])
similQd=np.dot(Qtemp,V[0:reductie,:])/np.dot(np.abs(Qtemp),np.abs(V[0:reductie,:]))*100
for xyz in range (len(V)):
    if similQd[xyz]>20:
        disname=dia.iloc[[xyz-1]]
        disname.ix[:,'index'] =similQd[xyz]
        dise=dise.append(disname)
print(dise.sort_values(('index'),ascending=0))

In [ ]:
# ******************************************* PRINT LIST OF relevant symptoms
syme=dia[dia['_id']==0]
similQs=np.dot(Qtemp,US.T)/np.dot(np.abs(Qtemp),np.abs(US.T))*100
for xyz in range (len(V)):
    if similQs[xyz]>20:
        symname=sym.iloc[[xyz-1]]
        symname.ix[:,'index']=similQs[xyz]
        syme=syme.append(symname)
print(syme.sort_values(('index'),ascending=0))